In [ ]:

!pip install -q pyautogen openai python-dotenv gradio google-generativeai "ag2[gemini]"

In [ ]:

import warnings
warnings.filterwarnings("ignore")

import os
import autogen
import gradio as gr
from openai import OpenAI  
from dotenv import load_dotenv
from IPython.display import display, Markdown
import random  
import google.generativeai as genai  # Import the Google library

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")  # Load the Google API key

print("Setup Complete: Libraries installed and API keys loaded.")

In [ ]:

def print_markdown(text):
    display(Markdown(text))

In [ ]:

config_list_openai = [
    {
        "model": "gpt-4o-mini",
        "api_key": openai_api_key,
    }
]

llm_config_openai = {
    "config_list": config_list_openai,
    "temperature": 0.7,  
    "timeout": 120,
}

In [ ]:
cmo_prompt = """You are the Chief Marketing Officer (CMO) of a new shoe brand (sustainable).
Provide high-level strategy, define target audiences, and guide the Marketer. Focus on the big picture. Be concise."""

In [ ]:
brand_marketer_prompt = """You are the Brand Marketer for the shoe brand. Brainstorm creative, specific campaign ideas (digital, content, experiences).
Focus on tactics and details. Suggest KPIs for your ideas."""

In [ ]:

cmo_agent_openai = autogen.ConversableAgent(
    name = "Chief_Marketing_Officer_OpenAI",
    system_message = cmo_prompt,
    llm_config = llm_config_openai,  
    human_input_mode = "NEVER")

print(f"Agent '{cmo_agent_openai.name}' created (using OpenAI).")

In [ ]:

brand_marketer_agent_openai = autogen.ConversableAgent(
    name = "Brand_Marketer_OpenAI",
    system_message = brand_marketer_prompt,
    llm_config = llm_config_openai,  # Assign the same OpenAI config
    human_input_mode = "NEVER")

print(f"Agent '{brand_marketer_agent_openai.name}' created (using OpenAI).")

In [ ]:
initial_task_message = """
Context: We're launching a new sustainable shoe line and need campaign ideas
Instruction: Brainstorm a campaign concept with specific elements
Input: Our sustainable, futuristic shoe brand needs marketing direction
Output: A concise campaign concept with the following structure:
Brand Marketer, let's brainstorm initial campaign ideas for our new sustainable shoe line.
Give me a distinct campaign concept. Outline: core idea, target audience, primary channels, and 1-2 KPIs. Keep it concise. Try to arrive at a final answer in 2-3 turns.
"""

print("--- Starting Agent Conversation (OpenAI Only) ---")
print("Chief Marketing Officer (OpenAI) initiating chat with Brand Marketer (OpenAI). Max Turns = 4")
print("--------------------------------------------------")


chat_result_openai_only = cmo_agent_openai.initiate_chat(
    recipient = brand_marketer_agent_openai, message = initial_task_message, max_turns = 4
)

print("--------------------------------------------------")
print("--- Conversation Ended (OpenAI Only) ---")

In [ ]:
def print_chat_history(chat_result):
    """Any chat result object has a chat_history attribute that contains the conversation history.
    This function prints the conversation history in a readable format.
    """
    for i in chat_result.chat_history:  
        print_markdown(i['name'])
        print("_"*100)
        print_markdown(i['content'])
        print("_"*100)


In [ ]:

print_chat_history(chat_result_openai_only)

In [ ]:

config_list_gemini = [
    {
        "model": "gemini-2.0-flash", 
        "api_key": google_api_key,
        "api_type": "google", 
    }
]

llm_config_gemini = {
    "config_list": config_list_gemini,
    "temperature": 0.6,  
    "timeout": 120,
}

In [ ]:

cmo_agent_gemini = autogen.ConversableAgent(
    name = "Chief_Marketing_Officer_Gemini",
    system_message = cmo_prompt,
    llm_config = llm_config_gemini,  
    human_input_mode = "NEVER")

brand_marketer_agent_openai_mixed = autogen.ConversableAgent(
    name = "Brand_Marketer_OpenAI",  # Keep name consistent if desired, or update
    system_message = brand_marketer_prompt,
    llm_config = llm_config_openai,  # Assign the OpenAI config!
    human_input_mode = "NEVER")

print(f"Agent '{cmo_agent_gemini.name}' created (using Google Gemini).")
print(f"Agent '{brand_marketer_agent_openai_mixed.name}' created (using OpenAI).")

Let's initiate the same chat as before, but now using our **mixed-model team**: the Gemini-powered Chief Marketing Officer and the OpenAI's GPT-powered Brand Marketer.

Observe the conversation flow. Do you notice any subtle differences in style or focus compared to the OpenAI-only chat, potentially due to the different underlying models?


In [ ]:


print("--- Starting Agent Conversation (Multi-Model: Gemini + OpenAI) ---")
print("Chief Marketing Officer (Gemini) initiating chat with Brand Marketer (OpenAI). Max Turns = 4")
print("------------------------------------------------------------------")

chat_result_multi_model = cmo_agent_gemini.initiate_chat(
    recipient = brand_marketer_agent_openai_mixed,  # Target the OpenAI marketer
    message = initial_task_message,
    max_turns = 4)

print("------------------------------------------------------------------")
print("--- Conversation Ended (Multi-Model) ---")


In [ ]:
print_chat_history(chat_result_multi_model)

In [ ]:
# Create the User Proxy Agent (Represents You)
user_proxy_agent = autogen.UserProxyAgent(
    name = "Human_User_Proxy",
    human_input_mode = "ALWAYS",  # Prompt user for input until 'exit'
    max_consecutive_auto_reply = 1,
    is_termination_msg = lambda x: x.get("content", "").rstrip().lower() in ["exit", "quit", "terminate"],
    code_execution_config = False,
    system_message = "You are the human user interacting with a multi-model AI team (Gemini CMO, OpenAI Marketer). Guide the brainstorm. Type 'exit' to end.",
)

print(f"Agent '{user_proxy_agent.name}' created for HIL with multi-model team.")

print("--- Starting Human-in-the-Loop (HIL) Conversation (Multi-Model) ---")
print("You will interact with Gemini CMO and OpenAI Marketer. Type 'exit' to end.")
print("---------------------------------------------------------------------")

In [ ]:

cmo_agent_gemini.reset()  # Reset Gemini CMO
brand_marketer_agent_openai_mixed.reset()  # Reset OpenAI Marketer
user_proxy_agent.reset()

In [ ]:
from autogen import GroupChat, GroupChatManager

groupchat = GroupChat(
    agents = [user_proxy_agent, cmo_agent_gemini, brand_marketer_agent_openai],  # List of agents participating in the group chat
    messages = [ ],  # Initialize with empty message history
    max_round = 20,  # Optional: Limits how many conversation rounds can occur before terminating
)

group_manager = GroupChatManager(groupchat = groupchat, llm_config = llm_config_openai)  # Uses OpenAI's LLM to manage the conversation

In [ ]:

group_chat_result = group_manager.initiate_chat(
    recipient = user_proxy_agent,  # Start by talking to the Gemini CMO
    message = """Hello team!!""",
)

print("---------------------------------------------------------------------")
print("--- Conversation Ended (Human terminated or Max Turns) ---")

In [ ]:
print_chat_history(group_chat_result)

In [ ]:
social_media_prompt = """You are the Social Media Strategist for a new sustainable shoe brand. 
            Develop creative social media posts, suggest platforms, posting times, and engagement strategies. 
            Work with the Brand Marketer and CMO."""

In [ ]:
# Let's create our social media strategist Agent - Using OpenAI for now
social_media_agent_openai = autogen.ConversableAgent(
    name = "Social_Media_OpenAI",
    system_message = social_media_prompt,
    llm_config = llm_config_openai,  # Assign the OpenAI config
    human_input_mode = "NEVER")

print(f"Agent '{social_media_agent_openai.name}' created (using OpenAI).")

In [ ]:
chat_result_openai_only = cmo_agent_openai.initiate_chat(
    recipient = brand_marketer_agent_openai, message = initial_task_message, max_turns = 6)


In [ ]:
# Configuration for Claude Agent
config_list_claude = [
    {
        "model": "claude-3-5-sonnet-20240620",
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

llm_config_claude = {
    "config_list": config_list_claude,
    "temperature": 0.6,  # Use a slightly higher temp for creative marketing ideas
    "timeout": 120,
}

In [ ]:
social_media_prompt = """You are the Social Media Strategist for a new sustainable shoe brand. 
            Develop creative social media posts, suggest platforms, posting times, and engagement strategies. 
            Work with the Brand Marketer and CMO."""

In [ ]:
# Let's create our social media strategist Agent using Claude
social_media_agent_claude = autogen.ConversableAgent(
    name = "Social_Media_Claude",
    system_message = social_media_prompt,
    llm_config = llm_config_claude,  # Assign the Claude config
    human_input_mode = "NEVER")

print(f"Agent '{social_media_agent_claude.name}' created (using Claude).")

In [ ]:
initial_task_message = """
Context: We're launching a new sustainable shoe line and need campaign ideas
Instruction: Brainstorm a campaign concept with specific elements
Input: Our sustainable, futuristic shoe brand needs marketing direction
Output: A concise campaign concept with the following structure:
Brand Marketer, let's brainstorm initial campaign ideas for our new sustainable shoe line.
Give me a distinct campaign concept. Outline: core idea, target audience, primary channels, and 1-2 KPIs. Keep it concise. Try to arrive at a final answer in 2-3 turns.
"""

In [ ]:

config_list_gemini = [
    {
        "model": "gemini-2.0-flash",  # Or "gemini-pro"
        "api_key": google_api_key,
        "api_type": "google",  # Specify the API type for Autogen's Google integration
    }
]

llm_config_gemini = {
    "config_list": config_list_gemini,
    "temperature": 1.6,  # Maybe slightly less randomness for strategic Chief Marketing Officer
    "timeout": 15,
}

In [ ]:

cmo_agent_gemini = autogen.ConversableAgent(
    name = "Chief_Marketing_Officer_Gemini",
    system_message = cmo_prompt,
    llm_config = llm_config_gemini,  # Assign the Gemini config!
    human_input_mode = "NEVER")


In [ ]:

config_list_openai = [
    {
        "model": "gpt-4o-mini",
        "api_key": openai_api_key,
    }
]

llm_config_openai = {
    "config_list": config_list_openai,
    "temperature": 1.6,  # Use a slightly higher temp for creative marketing ideas
    "timeout": 120,
}

In [ ]:

brand_marketer_agent_openai_mixed = autogen.ConversableAgent(
    name = "Brand_Marketer_OpenAI",  # Keep name consistent if desired, or update
    system_message = brand_marketer_prompt,
    llm_config = llm_config_openai,  # Assign the OpenAI config!
    human_input_mode = "NEVER")


In [ ]:
print("--- Starting Agent Conversation (Multi-Model: Gemini + OpenAI) ---")
print("Chief Marketing Officer (Gemini) initiating chat with Brand Marketer (OpenAI). Max Turns = 4")
print("------------------------------------------------------------------")

chat_result_multi_model = cmo_agent_gemini.initiate_chat(
    recipient = brand_marketer_agent_openai_mixed,  # Target the OpenAI marketer
    message = initial_task_message,
    max_turns = 6)

print("------------------------------------------------------------------")
print("--- Conversation Ended (Multi-Model) ---")


In [ ]:

groupchat = GroupChat(
       agents = [user_proxy_agent, cmo_agent_gemini, brand_marketer_agent_openai, social_media_agent_claude],
       messages = [],
       max_round= 10)

group_manager = GroupChatManager(groupchat = groupchat, llm_config = llm_config_openai)  # Uses OpenAI's LLM to manage the conversation

In [ ]:
group_chat_result = group_manager.initiate_chat(
       recipient = user_proxy_agent,
       message = """Let's develop a TikTok campaign for our shoes targeting Gen Z.
       CMO (Gemini): Please outline the strategic goals.
       Brand Marketer (OpenAI): Suggest a campaign hook.
       Social Media Strategist (Claude): Provide a social Media plan""",
       max_turns = 10)